In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import aicspylibczi as aplc
import xml.etree.ElementTree as ET


In [2]:
os.getcwd()

'/local/workdir/bmg224/jupyter'

In [4]:
cluster = ''
workdir = '/workdir/bmg224/hiprfish/zen_tilted_convex_hull/20240228_tiltest'
os.chdir(cluster + workdir)
os.getcwd()

'/local/workdir/bmg224/hiprfish/zen_tilted_convex_hull/20240228_tiltest'

In [5]:
fn = '20240228_tilttest.czi'

In [5]:
# javabridge.start_vm(class_path=bioformats.JARS)

In [ ]:
# raw = bioformats.load_image(fn, series=0)
# raw.shape

In [ ]:
# ims_meta_str = bioformats.get_omexml_metadata(path=fn)
# ims_meta = ET.fromstring(ims_meta_str)

In [15]:
# for elem in ims_meta.iter():
#     if isinstance(elem.text,str):
#             # if 'Index' in elem.text:
#             print(elem.tag, elem.text)

{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key 20240228_tilttest #05 Positions|Series 05| #10
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Value 4
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key Information|Image|S|Scene|Index
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Value [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key Experiment|AcquisitionBlock|Laser|LaserPower
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Value [0.0050000000000000001, 0.02, 0.024]
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key Experiment|AcquisitionBlock|MultiTrackSetup|TrackSetup|Attenuator|Polarization_Stokes3
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Value [0]
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key Experiment|AcquisitionBlock|MultiTrackSetup|TrackSetup|Attenuator|Polarization_Stokes2
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Value [0]
{http://www.openmicroscopy.org/Schemas/OME/2016-06}Key E

In [13]:
def get_resolution(fn, dim='X'):
    czi = aplc.CziFile(fn)
    for n in czi.meta.iter():
        if 'Scaling' in n.tag:
            if dim in n.tag:
                resolution = float(n.text)
    return resolution

def get_metadata_value(fn, search_term):
    czi = aplc.CziFile(fn)
    results = []
    for n in czi.meta.iter():
        if search_term in n.tag:
            results.append(n.text)
    return results

def load_raw(fn, m, mtype):
    czi = aplc.CziFile(fn)
    if mtype == 'M':
        raw, sh = czi.read_image(M=m)
    elif mtype == 'S':
        raw, sh = czi.read_image(S=m)
    else:
        raw, sh = czi.read_image()
    return raw

def reshape_aics_image(m_img):
    '''
    Given an AICS image with just XY and CHannel,
    REshape into shape (X,Y,C)
    '''
    img = np.squeeze(m_img)
    img = np.transpose(img, (1,2,0))
    return img

In [14]:
ntiles = int(get_metadata_value(fn, 'SizeS')[0])
ntiles

12

In [15]:
tif_dir = 'scenes_tif'
if not os.path.exists(tif_dir): 
    os.makedirs(tif_dir)


In [23]:
im_sum.shape

(2048, 2048)

In [24]:
for s in range(ntiles):
    raw = load_raw(fn, s, 'S')
    raw = reshape_aics_image(raw)
    im_sum = np.sum(raw, axis=2).astype(np.uint8)
    im = Image.fromarray(im_sum)
    im.save(tif_dir + '/' + str(s) + '.tif')

In [ ]:
im = Image.fromarray(raw)

## Adjust coordinates

In [14]:
coord_fn = 'test_outputs/xyz_coordinates.csv'
coords = pd.read_csv(coord_fn, header=None)
coords.columns = ["x", "y", "z"]
coords

,x,y,z
0,-3610.867,-4262.918,2320.802
1,-3610.867,-4060.918,2321.110
2,-3408.867,-4464.918,2321.751
3,-3408.867,-4262.918,2322.060
4,-3408.867,-4060.918,2322.368
5,-3206.867,-4464.918,2323.009
6,-3206.867,-4262.918,2323.317
7,-3206.867,-4060.918,2323.626
8,-3004.867,-4262.918,2324.575
9,-3004.867,-4060.918,2324.883


Convert location to pixels

In [32]:
res_umpix = 0.1037832

In [29]:
coords["xpix"] = coords["x"] / res_umpix
coords["ypix"] = coords["y"] / res_umpix
coords

,x,y,z,yinv,xpix,ypix
0,-3610.867,-4262.918,2320.802,4262.918,-34792.403780,-41075.222194
1,-3610.867,-4060.918,2321.110,4060.918,-34792.403780,-39128.857079
2,-3408.867,-4464.918,2321.751,4464.918,-32846.038665,-43021.587309
3,-3408.867,-4262.918,2322.060,4262.918,-32846.038665,-41075.222194
4,-3408.867,-4060.918,2322.368,4060.918,-32846.038665,-39128.857079
5,-3206.867,-4464.918,2323.009,4464.918,-30899.673550,-43021.587309
6,-3206.867,-4262.918,2323.317,4262.918,-30899.673550,-41075.222194
7,-3206.867,-4060.918,2323.626,4060.918,-30899.673550,-39128.857079
8,-3004.867,-4262.918,2324.575,4262.918,-28953.308435,-41075.222194
9,-3004.867,-4060.918,2324.883,4060.918,-28953.308435,-39128.857079


## Write stitching coords file

In [30]:
lines = ['dim = 2']
for s in range(12):
    xy = tuple(coords.loc[s,['xpix','ypix']].values)
    line = str(s) + '.tif;;' + str(xy)
    lines.append(line)
for l in lines:
    print(l)

dim = 2
0.tif;;(-34792.40378018793, -41075.22219395817)
1.tif;;(-34792.40378018793, -39128.857078987734)
2.tif;;(-32846.03866521749, -43021.58730892861)
3.tif;;(-32846.03866521749, -41075.22219395817)
4.tif;;(-32846.03866521749, -39128.857078987734)
5.tif;;(-30899.673550247055, -43021.58730892861)
6.tif;;(-30899.673550247055, -41075.22219395817)
7.tif;;(-30899.673550247055, -39128.857078987734)
8.tif;;(-28953.308435276616, -41075.22219395817)
9.tif;;(-28953.308435276616, -39128.857078987734)
10.tif;;(-27006.943320306178, -41075.22219395817)
11.tif;;(-27006.943320306178, -39128.857078987734)


In [31]:
coords_yinv_fn = 'scenes_tif/stitch_coords_pix.txt'
with open(coords_yinv_fn, 'w') as f:
    for l in lines:
        f.write(l + '\n')
print('Wrote:', coords_yinv_fn)

Wrote: scenes_tif/stitch_coords_yinv.txt
